In [9]:
import requests as re
import os
from IPython.display import JSON  
import pandas as pd


In [10]:
app_id = os.environ["YELP_CLIENT_ID"]
app_key = os.environ["YELP_API_KEY"]
# url_append = f'?app_id={app_id}&app_key={app_key}' 

headers= {'Authorization': 'Bearer {}'.format(app_key)}

In [11]:
# YELP URL
url = "https://api.yelp.com/v3/businesses/search"

params = { 'limit': 50,
           'term':'restaurants, bars',
           'longitude': -114.0593,
            'latitude': 51.0478,
          'radius': 5000,
          #attributes': 'open_to_all'
         }

In [12]:
# Send request to API
res = re.get(url, headers=headers, params=params,timeout=5)

In [13]:
# check if the request was successful
res.status_code

200

In [14]:
# We can export the information that was returned using method .json()
data_dict =res.json()
#data_dict
JSON(data_dict)

<IPython.core.display.JSON object>

In [15]:
import sqlite3
from sqlite3 import Error


In [16]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection    

In [17]:
connection = create_connection("downtown_calgary_POI.sqlite")

Connection to SQLite DB successful


In [18]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [26]:
#open JSON data
df = pd.json_normalize(data_dict, "businesses") 
#remove come columns
df1= df.drop(columns=["categories","id","location.address2","location.address3","transactions", "location.display_address"])
#df1

In [28]:
#create table in sql
df_sql = df1.to_sql('yelp_businesses', connection, if_exists='fail', index=True)

In [311]:
#Selecting records
def execute_read_query(connection, query):
    cursor= connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result=cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [313]:
yelp = "SELECT * from yelp_businesses"
yelp_bus = execute_read_query(connection, yelp)

#for bus in yelp_bus:
 #   print(bus)
    

In [29]:
top_10_POI = df1.sort_values(by=["rating"], ascending= False)
top_10_POI.head(1)

,alias,name,image_url,is_closed,url,review_count,rating,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.city,location.zip_code,location.country,location.state,price
0,paper-lantern-calgary-2,Paper Lantern,https://s3-media3.fl.yelpcdn.com/bphoto/sGNVqJ...,False,https://www.yelp.com/biz/paper-lantern-calgary...,5,5.0,+14034577765,+1 403-457-7765,386.982967,51.050879,-114.061746,115 - 2 Avenue SE,Calgary,T2G 0B2,CA,AB,NaN
